In [ ]:
import pandas as pd
import re

ratings_movies = pd.read_csv('data/ratings_movies.csv')
ratings_movies['date'] = pd.to_datetime(ratings_movies['date'])

def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

In [14]:
# Создайте в таблице новый признак year_release, который соответствует году выпуска фильма.
# У скольких фильмов не указан год их выпуска?

ratings_movies['year_release'] = ratings_movies['title'].apply(get_year_release)
ratings_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Unnamed: 0    100836 non-null  int64         
 1   userId        100836 non-null  int64         
 2   movieId       100836 non-null  int64         
 3   rating        100836 non-null  float64       
 4   date          100836 non-null  datetime64[ns]
 5   title         100836 non-null  object        
 6   genres        100836 non-null  object        
 7   year_release  100818 non-null  float64       
 8   year_rating   100836 non-null  int32         
dtypes: datetime64[ns](1), float64(2), int32(1), int64(3), object(2)
memory usage: 6.5+ MB


In [ ]:
# Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?

ratings_movies[ratings_movies['year_release'] == 1999].groupby('title')['rating'].mean().sort_values()

In [ ]:
# Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, 
# получило наименьшую среднюю оценку (rating)?

ratings_movies[ratings_movies['year_release'] == 2010].groupby('genres')['rating'].mean().sort_values()

In [ ]:
# Какой пользователь (userId) посмотрел наибольшее количество различных 
# (уникальных) комбинаций жанров (genres) фильмов?

ratings_movies.groupby('userId')['genres'].nunique().sort_values()

In [ ]:
# Найдите пользователя, который выставил наименьшее количество оценок, 
# но его средняя оценка фильмам наибольшая.

ratings_movies.groupby('userId')['rating'].agg(['count','max']).sort_values(
    by=['count','max'],
    ascending=[True,False]
)

In [ ]:
# Найдите сочетание жанров (genres) за 2018 год, которое имеет наибольший средний 
# рейтинг (среднее по столбцу rating), и при этом число выставленных ему оценок 
# (количество значений в столбце rating) больше 10.

mask = ratings_movies['year_release'] == 2018

ratings_movies[mask].groupby('genres')['rating'].agg(['mean','count']).sort_values(
    by=['mean','count'],
    ascending=[False, True]
)

In [51]:
ratings_movies['year_rating'] = ratings_movies['date'].dt.year

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

mask = ratings_movies['genres'] == 'Comedy'

res = ratings_movies.pivot_table(
    values='rating',
    index='genres',
    columns='year_rating',
    aggfunc='mean'
)
display(res[res.index == 'Comedy'])

year_rating,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
genres,,,,,,,,,,,,,,,,,,,,,,,
Comedy,3.228571,3.409091,3.0,3.606061,3.141291,3.318408,3.198556,3.120066,3.356877,2.963325,3.347534,2.928187,3.426667,3.273292,3.179825,3.232877,3.478477,3.489474,3.271429,3.106183,3.373431,2.852668,3.121296
